Этот файл описывает процесс преобразования данных для работы с сервисом AlphaMountain, который использует числовые коды вместо текстовых категорий. Коды категорий используются как ключи для соединения с таблицей, содержащей информацию о категориях.


1. Загрузка данных
Загружаются два файла: categorized_new_url_task_2_final_4.csv, который содержит URL-адреса, и am_mapping.csv, в котором хранится информация о категориях. Для этого используется библиотека pandas.

2. Переименование столбцов
В таблице категорий переименовываются столбцы. Столбцы id и name переименовываются в Category_Code_1, alpha_1 и Category_Code_2, alpha_2. Это необходимо для того, чтобы привести данные к нужному формату перед их объединением.

3. Извлечение категорий
Функция extract_category_codes извлекает список категорий для каждого URL. Эти данные содержатся в столбце Category. Если формат данных неправильный, функция возвращает пустой список.

4. Создание новых столбцов
Для каждого значения в списке категорий создаются новые столбцы, которые соответствуют отдельным категориям. Если категория отсутствует, в соответствующем столбце будет установлено значение None.

5. Объединение данных
После того как столбцы категорий были созданы, таблица results_month_api объединяется с таблицей category_api_v_1 и category_api_v_2 по соответствующим категориям, что позволяет добавить информацию о названиях категорий в исходные данные.

6. Сохранение результатов
Итоговые данные сохраняются в новый файл категоризация alpha_4.csv, который содержит URL-адреса и соответствующие им категории.

1. Загрузка библиотек

In [1]:
import pandas as pd

2. Выгрузка данных

In [2]:
results_month_api = pd.read_csv('categorized_new_url_task_2_final_4.csv')

In [3]:
category_api = pd.read_csv('am_mapping.csv')

3. Предобработка данных 

In [5]:
# Переименование колонок сразу в двух DataFrame в две строки
category_api_v_1 = category_api.rename(columns={'id': 'Category_Code_1', 'name': 'alpha_1'})
category_api_v_2 = category_api.rename(columns={'id': 'Category_Code_2', 'name': 'alpha_2'})

In [ ]:
4. Извлечение категорий

Этот код извлекает категории из столбца Category в таблице results_month_api, применяя функцию для преобразования строкового представления категорий в список. Затем для каждой категории создаются новые столбцы с уникальными кодами, соответствующими каждому URL. После этого результат выводится для проверки, а итоговые данные могут быть сохранены в новый файл, если необходимо.

In [6]:
# Функция для извлечения всех значений категории
def extract_category_codes(category_data):
    try:
        # Преобразуем строку в словарь, если это строка
        if isinstance(category_data, str):
            category_data = eval(category_data)
        # Получаем список категорий
        categories = category_data.get('category', {}).get('categories', [])
        return categories
    except (AttributeError, TypeError):
        return []  # Возвращаем пустой список, если формат данных не соответствует ожиданиям

# Применение функции к столбцу Category и создание новых столбцов для каждого значения
category_codes = results_month_api['Category'].apply(extract_category_codes)

# Создание новых столбцов в зависимости от количества категорий
for i in range(category_codes.str.len().max()):
    results_month_api[f'Category_Code_{i+1}'] = category_codes.apply(lambda x: x[i] if i < len(x) else None)

# Проверка результата
print(results_month_api[['url', 'Category'] + [f'Category_Code_{i+1}' for i in range(category_codes.str.len().max())]].head())

# Сохранение в новый файл (при необходимости)
# results_month_api.to_csv('results_month_api_with_codes.csv', index=False)


                             url  \
0                www.tiktokv.com   
1                     www.com.co   
2  www.find-packing-services.com   
3           www.tradedoubler.com   
4                www.mozilla.net   

                                            Category  Category_Code_1  \
0  {'version': 1, 'status': {'category': 'Success...               18   
1  {'version': 1, 'status': {'category': 'Success...               39   
2  {'version': 1, 'status': {'category': 'Success...                0   
3  {'version': 1, 'status': {'category': 'Success...                2   
4  {'version': 1, 'status': {'category': 'Success...               34   

   Category_Code_2  Category_Code_3  
0             67.0             79.0  
1              NaN              NaN  
2              NaN              NaN  
3              9.0             34.0  
4              NaN              NaN  


Этот код выполняет слияние таблиц: сначала таблица results_month_api объединяется с таблицей category_api_v_1 по столбцу Category_Code_1 с использованием левого соединения (left join), а затем результат объединяется с таблицей category_api_v_2 по столбцу Category_Code_2, также с левым соединением. Это позволяет добавить соответствующие данные из таблиц категорий в таблицу с результатами.

In [7]:
results_month_api_temp = results_month_api.merge(category_api_v_1, on = 'Category_Code_1', how = 'left')
results = results_month_api_temp.merge(category_api_v_2, on = 'Category_Code_2', how = 'left')


In [8]:
results = results[['url', 'alpha_1','alpha_2']]

In [9]:
results.to_csv('категоризация alpha_4.csv', index=False)

In [10]:
results

,url,alpha_1,alpha_2
0,www.tiktokv.com,Entertainment,Social Networking
1,www.com.co,Malicious,NaN
2,www.find-packing-services.com,Unrated,NaN
3,www.tradedoubler.com,Ads/Analytics,Business/Economy
4,www.mozilla.net,Information Technology,NaN
...,...,...,...
429,www.lordfilm.art,Unrated,NaN
430,www.vega-factory.ru,Business/Economy,NaN
431,23.227.151.122,Information Technology,NaN
432,www.championat.com,Sports,NaN


В результате выполнения кода создается таблица, которая включает информацию о URL-адресах, а также категории, полученные из исходных данных с использованием кодов категорий. В новой таблице объединяются значения из столбцов Category_Code_1 и Category_Code_2 с соответствующими категориями из таблицы category_api_v_1 и category_api_v_2. Этот процесс позволяет более детально классифицировать данные и подготовить их для дальнейшего анализа или использования в других системах. Итоговая таблица сохраняется в файл, который можно использовать для дальнейшей обработки или отчетности.